In [16]:
# === IMPORTACIÓN DE LIBRERÍAS ===
import os                   
import requests             
import pandas as pd         
import json                 
from dotenv import load_dotenv  

In [17]:
load_dotenv()
URL = "https://cramer.buk.cl/apidocs#!/Inasistencias/get_absences_absence"

In [18]:
# === AUTENTICACIÓN Y HEADERS ===
# Obtiene el token de autenticación desde variable de entorno (más seguro)
TOKEN = os.getenv("TOKEN")
if not TOKEN:
    # Si no encuentra el token, termina el programa con mensaje de error
    raise SystemExit("ERROR: Define la variable de entorno BUK_AUTH_TOKEN con tu token")

print("✅ Token encontrado exitosamente")

# Configura los headers que se enviarán en cada petición HTTP
HEADERS = {
    "auth_token": TOKEN,           # Token de autenticación para la API
    "Accept": "application/json",  # Le dice al servidor que esperamos respuesta en JSON
    "Content-Type": "application/json"  # Especifica el tipo de contenido que enviamos
}

✅ Token encontrado exitosamente


In [19]:
# === FUNCIÓN MEJORADA PARA MÚLTIPLES ENDPOINTS CON FILTRO DE FECHAS ===
from datetime import datetime, date

def fetch_data_from_multiple_endpoints(fecha_inicio="2025-05-01", fecha_fin=None):
    """
    Obtiene datos de múltiples endpoints de BUK con filtro de fechas y los combina.
    Maneja paginación automáticamente para cada endpoint.
    
    Args:
        fecha_inicio (str): Fecha de inicio en formato 'YYYY-MM-DD'. Por defecto '2025-05-01'
        fecha_fin (str): Fecha de fin en formato 'YYYY-MM-DD'. Si es None, usa fecha actual
    
    Returns:
        tuple: (datos_combinados, datos_por_endpoint, info_fechas)
    """
    
    # Procesar fechas
    if fecha_fin is None:
        fecha_fin = date.today().strftime('%Y-%m-%d')
    
    # Validar formato de fechas
    try:
        datetime.strptime(fecha_inicio, '%Y-%m-%d')
        datetime.strptime(fecha_fin, '%Y-%m-%d')
    except ValueError as e:
        raise ValueError(f"Formato de fecha inválido. Usa 'YYYY-MM-DD'. Error: {e}")
    
    # Información de fechas para retornar
    info_fechas = {
        'fecha_inicio': fecha_inicio,
        'fecha_fin': fecha_fin,
        'fecha_extraccion': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Configuración de endpoints
    base_url = "https://cramer.buk.cl/api/v1/chile"
    endpoints = {
        "inasistencias": "/absences/absence",
        "licencias": "/absences/licence", 
        "permisos": "/absences/permission"
    }
    
    # Diccionario para almacenar datos de cada endpoint
    all_data = {}
    combined_records = []
    
    print("INICIANDO EXTRACCIÓN DE MÚLTIPLES ENDPOINTS BUK CON FILTRO DE FECHAS")
    print("=" * 70)
    print(f"Período: {fecha_inicio} a {fecha_fin}")
    print("=" * 70)
    
    # Iterar por cada endpoint
    for endpoint_name, endpoint_path in endpoints.items():
        print(f"\nPROCESANDO ENDPOINT: {endpoint_name.upper()}")
        print(f"URL: {base_url}{endpoint_path}")
        print("-" * 50)
        
        # Construir URL completa con parámetros de fecha
        url = f"{base_url}{endpoint_path}"
        records_from_endpoint = []
        page_count = 0
        
        # Bucle de paginación para cada endpoint
        while url:
            try:
                print(f"📄 Página {page_count + 1}...")
                
                # Preparar parámetros de la petición con filtro de fechas
                params = {
                    'from': fecha_inicio,
                    'to': fecha_fin
                }
                
                # Realizar petición con parámetros de fecha
                response = requests.get(url, headers=HEADERS, params=params, timeout=15)
                response.raise_for_status()
                
                # Procesar respuesta
                json_data = response.json()
                
                # Extraer datos (estructura típica de BUK)
                page_data = json_data.get("data", [])
                
                if not page_data:
                    print(f"   ℹ️ No hay datos en esta página para el período especificado")
                    break
                
                # Agregar metadatos a cada registro
                for record in page_data:
                    if isinstance(record, dict):
                        record['_source_endpoint'] = endpoint_name
                        record['_source_url'] = f"{base_url}{endpoint_path}"
                        record['_fecha_extraccion'] = info_fechas['fecha_extraccion']
                
                records_from_endpoint.extend(page_data)
                print(f"   ✅ {len(page_data)} registros obtenidos (período: {fecha_inicio} a {fecha_fin})")
                
                # Buscar siguiente página
                pagination_info = json_data.get("pagination", {})
                next_url = pagination_info.get("next")
                
                if next_url:
                    # Mantener los parámetros de fecha en la siguiente página
                    url = next_url
                else:
                    url = None  # No hay más páginas
                
                page_count += 1
                
            except requests.HTTPError as e:
                print(f"❌ Error HTTP: {e}")
                print(f"📊 Status Code: {getattr(e.response, 'status_code', 'N/A')}")
                if hasattr(e.response, 'text'):
                    print(f"📄 Respuesta: {e.response.text[:300]}...")
                break
                
            except requests.RequestException as e:
                print(f"🌐 Error de conexión: {e}")
                break
                
            except ValueError as e:
                print(f"📋 Error JSON: {e}")
                if 'response' in locals():
                    print(f"📄 Respuesta: {response.text[:300]}...")
                break
        
        # Guardar resultados del endpoint
        all_data[endpoint_name] = records_from_endpoint
        combined_records.extend(records_from_endpoint)
        
        print(f"📊 Total {endpoint_name}: {len(records_from_endpoint)} registros en {page_count} páginas")
        print(f"📅 Período procesado: {fecha_inicio} a {fecha_fin}")
    
    # Resumen final
    print(f"\n" + "="*70)
    print(f"📈 RESUMEN DE EXTRACCIÓN COMPLETA CON FILTRO DE FECHAS:")
    print(f"📅 Período extraído: {fecha_inicio} a {fecha_fin}")
    print(f"🔢 Total endpoints procesados: {len(endpoints)}")
    
    for endpoint_name, records in all_data.items():
        print(f"   📊 {endpoint_name.capitalize()}: {len(records)} registros")
    
    print(f"🎯 TOTAL COMBINADO: {len(combined_records)} registros")
    print(f"⏰ Fecha de extracción: {info_fechas['fecha_extraccion']}")
    print(f"="*70)
    
    return combined_records, all_data, info_fechas

In [20]:
# === INFORMACIÓN DETALLADA DE LOS ENDPOINTS ===
print("📊 DETALLES DE LOS ENDPOINTS CONFIGURADOS")
print("=" * 60)

# Información sobre los endpoints (sin modificar la variable original)
base_url = "https://cramer.buk.cl/api/v1/chile"
endpoint_info = {
    "inasistencias": {
        "path": "/absences/absence",
        "method": "GET",
        "descripcion": "Obtiene registros de inasistencias/ausencias",
        "filtros_fecha": "Soporta parámetros 'from' y 'to'",
        "paginacion": "Sí (automática)"
    },
    "licencias": {
        "path": "/absences/licence",
        "method": "GET", 
        "descripcion": "Obtiene registros de licencias médicas",
        "filtros_fecha": "Soporta parámetros 'from' y 'to'",
        "paginacion": "Sí (automática)"
    },
    "permisos": {
        "path": "/absences/permission",
        "method": "GET",
        "descripcion": "Obtiene registros de permisos",
        "filtros_fecha": "Soporta parámetros 'from' y 'to'",
        "paginacion": "Sí (automática)"
    }
}

for nombre, info in endpoint_info.items():
    print(f"\n🔗 ENDPOINT: {nombre.upper()}")
    print(f"   📍 URL completa: {base_url}{info['path']}")
    print(f"   🔧 Método HTTP: {info['method']}")
    print(f"   📝 Descripción: {info['descripcion']}")
    print(f"   📅 Filtros de fecha: {info['filtros_fecha']}")
    print(f"   📄 Paginación: {info['paginacion']}")

print(f"\n✅ CONFIRMACIÓN:")
print(f"   • Todos los endpoints utilizan método GET")
print(f"   • Todos soportan filtros de fecha via query parameters")
print(f"   • La autenticación se realiza via header 'auth_token'")
print(f"   • Los parámetros de fecha van en la URL como ?from=YYYY-MM-DD&to=YYYY-MM-DD")

print(f"\n🌐 EJEMPLO DE URL COMPLETA CON FILTROS:")
ejemplo_fecha_inicio = "2025-05-01"
ejemplo_fecha_fin = "2025-08-25"
print(f"   {base_url}/absences/absence?from={ejemplo_fecha_inicio}&to={ejemplo_fecha_fin}")
print(f"\n📋 ESTRUCTURA DE LA PETICIÓN:")
print(f"   • Método: GET")
print(f"   • Headers: auth_token, Accept, Content-Type")
print(f"   • Query Params: from, to")
print(f"   • Timeout: 15 segundos")
print("=" * 60)

📊 DETALLES DE LOS ENDPOINTS CONFIGURADOS

🔗 ENDPOINT: INASISTENCIAS
   📍 URL completa: https://cramer.buk.cl/api/v1/chile/absences/absence
   🔧 Método HTTP: GET
   📝 Descripción: Obtiene registros de inasistencias/ausencias
   📅 Filtros de fecha: Soporta parámetros 'from' y 'to'
   📄 Paginación: Sí (automática)

🔗 ENDPOINT: LICENCIAS
   📍 URL completa: https://cramer.buk.cl/api/v1/chile/absences/licence
   🔧 Método HTTP: GET
   📝 Descripción: Obtiene registros de licencias médicas
   📅 Filtros de fecha: Soporta parámetros 'from' y 'to'
   📄 Paginación: Sí (automática)

🔗 ENDPOINT: PERMISOS
   📍 URL completa: https://cramer.buk.cl/api/v1/chile/absences/permission
   🔧 Método HTTP: GET
   📝 Descripción: Obtiene registros de permisos
   📅 Filtros de fecha: Soporta parámetros 'from' y 'to'
   📄 Paginación: Sí (automática)

✅ CONFIRMACIÓN:
   • Todos los endpoints utilizan método GET
   • Todos soportan filtros de fecha via query parameters
   • La autenticación se realiza via header 'auth_

In [21]:
# === EXTRACCIÓN COMBINADA CON RANGO DE FECHAS ===
print("🚀 EXTRAYENDO DATOS DE MÚLTIPLES ENDPOINTS BUK CON FILTRO DE FECHAS")
print("=" * 70)

try:
    # CONFIGURAR FECHAS AQUÍ:
    fecha_inicio = "2025-05-01"  
    fecha_fin = None             
    
    print(f" Configuración de fechas:")
    print(f"   • Fecha inicio: {fecha_inicio}")
    print(f"   • Fecha fin: {'Fecha actual' if fecha_fin is None else fecha_fin}")
    
    # Paso 1: Extraer de todos los endpoints CON RANGO DE FECHAS
    print(f"\n📡 Obteniendo datos desde múltiples APIs con filtro de fechas...")
    datos_combinados, datos_por_endpoint, info_fechas = fetch_data_from_multiple_endpoints(
        fecha_inicio=fecha_inicio, 
        fecha_fin=fecha_fin
    )
    
    if datos_combinados:
        print(f"\n✅ EXTRACCIÓN EXITOSA!")
        print(f"   📊 Total registros combinados: {len(datos_combinados)}")
        print(f"   📅 Período real extraído: {info_fechas['fecha_inicio']} a {info_fechas['fecha_fin']}")
        
        # Paso 2: Convertir a DataFrame combinado
        print(f"\n🔄 Creando DataFrame combinado...")
        df_combinado = pd.json_normalize(datos_combinados, sep="_")
        
        print(f"   📊 DataFrame combinado: {df_combinado.shape[0]} filas × {df_combinado.shape[1]} columnas")
        
        # Paso 3: Mostrar información por endpoint
        print(f"\n📋 DESGLOSE POR ENDPOINT:")
        if '_source_endpoint' in df_combinado.columns:
            conteo_por_endpoint = df_combinado['_source_endpoint'].value_counts()
            for endpoint, cantidad in conteo_por_endpoint.items():
                porcentaje = (cantidad / len(df_combinado)) * 100
                print(f"   • {endpoint.capitalize()}: {cantidad} registros ({porcentaje:.1f}%)")
        
        # Paso 4: Mostrar columnas disponibles
        print(f"\n📋 COLUMNAS DISPONIBLES ({len(df_combinado.columns)}):")
        columnas_por_fila = 3
        columnas = list(df_combinado.columns)
        
        for i in range(0, len(columnas), columnas_por_fila):
            fila_columnas = columnas[i:i+columnas_por_fila]
            print(f"   {i+1:2d}-{min(i+columnas_por_fila, len(columnas)):2d}. {' | '.join(fila_columnas)}")
        
        # Paso 5: Mostrar muestra de datos
        print(f"\n📄 MUESTRA DE DATOS COMBINADOS:")
        display(df_combinado.head())
        
        # Paso 6: Procesar campos de fecha
        print(f"\n📅 Procesando campos de fecha...")
        date_keywords = ["start_date", "end_date", "application_date", "created_at", "updated_at"]
        fechas_convertidas = 0
        for column in df_combinado.columns:
            if any(keyword in column.lower() for keyword in date_keywords):
                df_combinado[column] = pd.to_datetime(df_combinado[column], errors="coerce")
                fechas_convertidas += 1
                print(f"   ✅ Campo '{column}' convertido a fecha")
        
        print(f"   📊 Total campos de fecha convertidos: {fechas_convertidas}")
        
        # Paso 7: Validar rango de fechas en los datos
        if 'start_date' in df_combinado.columns:
            df_fechas_validas = df_combinado.dropna(subset=['start_date'])
            if len(df_fechas_validas) > 0:
                fecha_min_datos = df_fechas_validas['start_date'].min()
                fecha_max_datos = df_fechas_validas['start_date'].max()
                print(f"\n📈 VALIDACIÓN DE RANGO DE FECHAS:")
                print(f"   • Fecha mínima en datos: {fecha_min_datos.date()}")
                print(f"   • Fecha máxima en datos: {fecha_max_datos.date()}")
                print(f"   • Registros con fechas válidas: {len(df_fechas_validas):,}")
        
        # Paso 8: Guardar en variables globales
        globals()['df_buk_combinado'] = df_combinado
        globals()['datos_por_endpoint'] = datos_por_endpoint
        globals()['info_extraccion'] = info_fechas
        
        print(f"\n💾 DATOS GUARDADOS:")
        print(f"   • Variable 'df_buk_combinado': DataFrame con todos los datos")
        print(f"   • Variable 'datos_por_endpoint': Diccionario separado por endpoint")
        print(f"   • Variable 'info_extraccion': Información del rango de fechas usado")
        
        # Paso 9: Estadísticas finales
        print(f"\n📈 ESTADÍSTICAS FINALES:")
        print(f"   • Total registros: {len(df_combinado):,}")
        print(f"   • Total columnas: {len(df_combinado.columns)}")
        print(f"   • Período: {info_fechas['fecha_inicio']} a {info_fechas['fecha_fin']}")
        print(f"   • Memoria utilizada: {df_combinado.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        
        # Mostrar campos únicos por endpoint
        if '_source_endpoint' in df_combinado.columns:
            print(f"\n🔍 CAMPOS PRINCIPALES POR ENDPOINT:")
            for endpoint in df_combinado['_source_endpoint'].unique():
                df_endpoint = df_combinado[df_combinado['_source_endpoint'] == endpoint]
                campos_no_nulos = df_endpoint.notna().sum()
                campos_principales = campos_no_nulos[campos_no_nulos > 0].head(5)
                print(f"   • {endpoint.capitalize()}:")
                for campo, cantidad in campos_principales.items():
                    if campo not in ['_source_endpoint', '_source_url', '_fecha_extraccion']:
                        print(f"     - {campo}: {cantidad} registros")
        
    else:
        print("❌ No se pudieron obtener datos de ningún endpoint")
        print("💡 Posibles causas:")
        print("   • Tu token de autenticación no es válido")
        print("   • Las URLs de los endpoints no son correctas")
        print("   • No hay datos en el rango de fechas especificado")
        print("   • Problemas de conectividad")
        print(f"\n🔧 Sugerencias:")
        print(f"   • Verifica tu token en la variable de entorno")
        print(f"   • Prueba con un rango de fechas diferente")
        print(f"   • Revisa los logs de error arriba")
        
except Exception as e:
    print(f"💥 Error inesperado: {e}")
    print(f"   Tipo de error: {type(e).__name__}")
    import traceback
    print(f"   Detalles: {traceback.format_exc()}")
    
    # Información para debugging
    print(f"\n🔧 Para debugging:")
    print(f"   • Verifica que HEADERS esté definido")
    print(f"   • Confirma que tienes conexión a internet")
    print(f"   • Ejecuta las celdas anteriores primero")

🚀 EXTRAYENDO DATOS DE MÚLTIPLES ENDPOINTS BUK CON FILTRO DE FECHAS
 Configuración de fechas:
   • Fecha inicio: 2025-05-01
   • Fecha fin: Fecha actual

📡 Obteniendo datos desde múltiples APIs con filtro de fechas...
INICIANDO EXTRACCIÓN DE MÚLTIPLES ENDPOINTS BUK CON FILTRO DE FECHAS
Período: 2025-05-01 a 2025-08-26

PROCESANDO ENDPOINT: INASISTENCIAS
URL: https://cramer.buk.cl/api/v1/chile/absences/absence
--------------------------------------------------
📄 Página 1...
   ✅ 25 registros obtenidos (período: 2025-05-01 a 2025-08-26)
📄 Página 2...
   ✅ 11 registros obtenidos (período: 2025-05-01 a 2025-08-26)
📊 Total inasistencias: 36 registros en 2 páginas
📅 Período procesado: 2025-05-01 a 2025-08-26

PROCESANDO ENDPOINT: LICENCIAS
URL: https://cramer.buk.cl/api/v1/chile/absences/licence
--------------------------------------------------
📄 Página 1...
   ✅ 25 registros obtenidos (período: 2025-05-01 a 2025-08-26)
📄 Página 2...
   ✅ 25 registros obtenidos (período: 2025-05-01 a 2025-08

,id,start_date,end_date,days_count,day_percent,contribution_days,workday_stage,application_date,application_end_date,justification,...,medic_name,risk_type,sequela,incapacities_control,permission_type_id,permission_type_code,paid,time_measure,start_time,end_time
0,105184,2025-05-05,2025-05-05,1.0,1.0,0.0,full_working_day,2025-05-05,2025-05-05,Ausencia sin licencia médica,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,105185,2025-05-08,2025-05-08,1.0,1.0,0.0,full_working_day,2025-05-08,2025-05-08,Ausencia sin licencia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,105250,2025-05-02,2025-05-02,1.0,1.0,0.0,full_working_day,2025-05-02,2025-05-02,"Tramite personal, informado por Manuel Gamboa",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,105283,2025-05-08,2025-05-08,1.0,1.0,0.0,full_working_day,2025-05-08,2025-05-08,"Fallecimiento abuelita, se regaló 2 días y el ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105547,2025-05-11,2025-05-13,3.0,1.0,0.0,full_working_day,2025-05-14,2025-05-16,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



📅 Procesando campos de fecha...
   ✅ Campo 'start_date' convertido a fecha
   ✅ Campo 'end_date' convertido a fecha
   ✅ Campo 'application_date' convertido a fecha
   ✅ Campo 'application_end_date' convertido a fecha
   ✅ Campo 'created_at' convertido a fecha
   ✅ Campo 'updated_at' convertido a fecha
   📊 Total campos de fecha convertidos: 6

📈 VALIDACIÓN DE RANGO DE FECHAS:
   • Fecha mínima en datos: 2025-02-14
   • Fecha máxima en datos: 2025-08-26
   • Registros con fechas válidas: 234

💾 DATOS GUARDADOS:
   • Variable 'df_buk_combinado': DataFrame con todos los datos
   • Variable 'datos_por_endpoint': Diccionario separado por endpoint
   • Variable 'info_extraccion': Información del rango de fechas usado

📈 ESTADÍSTICAS FINALES:
   • Total registros: 234
   • Total columnas: 37
   • Período: 2025-05-01 a 2025-08-26
   • Memoria utilizada: 0.29 MB

🔍 CAMPOS PRINCIPALES POR ENDPOINT:
   • Inasistencias:
     - id: 36 registros
     - start_date: 36 registros
     - end_date: 36 

C:\Users\bgacitua\AppData\Local\Temp\ipykernel_22696\1560132489.py:59: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_combinado[column] = pd.to_datetime(df_combinado[column], errors="coerce")
C:\Users\bgacitua\AppData\Local\Temp\ipykernel_22696\1560132489.py:59: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_combinado[column] = pd.to_datetime(df_combinado[column], errors="coerce")


In [22]:
# === ANÁLISIS SEPARADO POR ENDPOINT ===
if 'datos_por_endpoint' in globals() and 'df_buk_combinado' in globals():
    print("ANÁLISIS DETALLADO POR ENDPOINT")
    print("=" * 50)
    
    for endpoint_name, registros in datos_por_endpoint.items():
        if registros:  # Solo si hay datos
            print(f"\nANÁLISIS: {endpoint_name.upper()}")
            print("-" * 30)
            
            # Crear DataFrame específico del endpoint
            df_endpoint = pd.json_normalize(registros, sep="_")
            
            print(f"Registros: {len(df_endpoint):,}")
            print(f"Columnas: {len(df_endpoint.columns)}")
            
            # Mostrar primeras 5 columnas más importantes
            print(f"Columnas principales:")
            for i, col in enumerate(df_endpoint.columns[:5], 1):
                valores_unicos = df_endpoint[col].nunique() if col in df_endpoint.columns else 0
                print(f"      {i}. {col} ({valores_unicos} valores únicos)")
            
            # Mostrar muestra
            print(f"Muestra de datos:")
            display(df_endpoint.head(3))
            
            # Guardar DataFrame individual
            globals()[f'df_{endpoint_name}'] = df_endpoint
            print(f"Guardado en variable: df_{endpoint_name}")
    
    print(f"\nANÁLISIS COMPLETADO")
    print(f"Variables disponibles:")
    print(f"   • df_buk_combinado: Todos los datos juntos")
    for endpoint_name in datos_por_endpoint.keys():
        if datos_por_endpoint[endpoint_name]:
            print(f"   • df_{endpoint_name}: Datos solo de {endpoint_name}")

else:
    print("Primero ejecuta la celda de extracción combinada")

ANÁLISIS DETALLADO POR ENDPOINT

ANÁLISIS: INASISTENCIAS
------------------------------
Registros: 36
Columnas: 19
Columnas principales:
      1. id (36 valores únicos)
      2. start_date (28 valores únicos)
      3. end_date (28 valores únicos)
      4. days_count (4 valores únicos)
      5. day_percent (1 valores únicos)
Muestra de datos:


,id,start_date,end_date,days_count,day_percent,contribution_days,workday_stage,application_date,application_end_date,justification,employee_id,status,created_at,updated_at,absence_type_id,absence_type_code,_source_endpoint,_source_url,_fecha_extraccion
0,105184,2025-05-05,2025-05-05,1.0,1,0.0,full_working_day,2025-05-05,2025-05-05,Ausencia sin licencia médica,6237,approved,2025-05-08T12:41:23.366-04:00,2025-05-08T12:41:23.366-04:00,2,ausencia,inasistencias,https://cramer.buk.cl/api/v1/chile/absences/ab...,2025-08-26 11:30:00
1,105185,2025-05-08,2025-05-08,1.0,1,0.0,full_working_day,2025-05-08,2025-05-08,Ausencia sin licencia,4918,approved,2025-05-08T12:42:26.143-04:00,2025-05-08T12:42:26.143-04:00,2,ausencia,inasistencias,https://cramer.buk.cl/api/v1/chile/absences/ab...,2025-08-26 11:30:00
2,105250,2025-05-02,2025-05-02,1.0,1,0.0,full_working_day,2025-05-02,2025-05-02,"Tramite personal, informado por Manuel Gamboa",4191,approved,2025-05-09T12:04:20.199-04:00,2025-05-09T12:04:20.199-04:00,2,ausencia,inasistencias,https://cramer.buk.cl/api/v1/chile/absences/ab...,2025-08-26 11:30:00


Guardado en variable: df_inasistencias

ANÁLISIS: LICENCIAS
------------------------------
Registros: 162
Columnas: 29
Columnas principales:
      1. id (162 valores únicos)
      2. start_date (87 valores únicos)
      3. end_date (88 valores únicos)
      4. days_count (23 valores únicos)
      5. day_percent (1 valores únicos)
Muestra de datos:


,id,start_date,end_date,days_count,day_percent,contribution_days,workday_stage,application_date,application_end_date,justification,...,format,licence_number,medic_rut,medic_name,risk_type,sequela,incapacities_control,_source_endpoint,_source_url,_fecha_extraccion
0,102676,2025-02-14,2025-05-08,84.0,1,0.0,full_working_day,2025-02-14,2025-05-08,,...,electronica,,None,,None,None,None,licencias,https://cramer.buk.cl/api/v1/chile/absences/li...,2025-08-26 11:30:00
1,104029,2025-04-02,2025-05-01,30.0,1,0.0,full_working_day,2025-04-02,2025-05-01,Licencia común,...,electronica,,None,,None,None,None,licencias,https://cramer.buk.cl/api/v1/chile/absences/li...,2025-08-26 11:30:00
2,104129,2025-04-05,2025-05-04,30.0,1,0.0,full_working_day,2025-04-05,2025-05-04,Licencia normal,...,electronica,,None,,None,None,None,licencias,https://cramer.buk.cl/api/v1/chile/absences/li...,2025-08-26 11:30:00


Guardado en variable: df_licencias

ANÁLISIS: PERMISOS
------------------------------
Registros: 36
Columnas: 23
Columnas principales:
      1. id (36 valores únicos)
      2. start_date (29 valores únicos)
      3. end_date (28 valores únicos)
      4. days_count (8 valores únicos)
      5. day_percent (2 valores únicos)
Muestra de datos:


,id,start_date,end_date,days_count,day_percent,contribution_days,workday_stage,application_date,application_end_date,justification,...,updated_at,permission_type_id,permission_type_code,paid,time_measure,start_time,end_time,_source_endpoint,_source_url,_fecha_extraccion
0,104986,2025-05-06,2025-05-07,2.0,1.0,0.0,full_working_day,2025-05-06,2025-05-07,Fallecimiento del abuelito,...,2025-05-06T09:13:56.325-04:00,9,permiso_fallecimiento_cramer,True,per_day,None,None,permisos,https://cramer.buk.cl/api/v1/chile/absences/pe...,2025-08-26 11:30:00
1,105448,2025-05-22,2025-05-22,1.0,1.0,0.0,full_working_day,2025-05-22,2025-05-22,Compensación día domingo 30-03-2025. Respaldo ...,...,2025-05-14T16:07:47.375-04:00,74,compensacion_por_viaje,True,per_day,None,None,permisos,https://cramer.buk.cl/api/v1/chile/absences/pe...,2025-08-26 11:30:00
2,105449,2025-05-12,2025-05-15,4.0,1.0,0.0,full_working_day,2025-05-12,2025-05-15,Fallecimiento hermana,...,2025-05-14T16:27:10.517-04:00,8,permiso_fallecimiento_legal,True,per_day,None,None,permisos,https://cramer.buk.cl/api/v1/chile/absences/pe...,2025-08-26 11:30:00


Guardado en variable: df_permisos

ANÁLISIS COMPLETADO
Variables disponibles:
   • df_buk_combinado: Todos los datos juntos
   • df_inasistencias: Datos solo de inasistencias
   • df_licencias: Datos solo de licencias
   • df_permisos: Datos solo de permisos


In [23]:
display(df_combinado)

,id,start_date,end_date,days_count,day_percent,contribution_days,workday_stage,application_date,application_end_date,justification,...,medic_name,risk_type,sequela,incapacities_control,permission_type_id,permission_type_code,paid,time_measure,start_time,end_time
0,105184,2025-05-05,2025-05-05,1.0,1.0,0.0,full_working_day,2025-05-05,2025-05-05,Ausencia sin licencia médica,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,105185,2025-05-08,2025-05-08,1.0,1.0,0.0,full_working_day,2025-05-08,2025-05-08,Ausencia sin licencia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,105250,2025-05-02,2025-05-02,1.0,1.0,0.0,full_working_day,2025-05-02,2025-05-02,"Tramite personal, informado por Manuel Gamboa",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,105283,2025-05-08,2025-05-08,1.0,1.0,0.0,full_working_day,2025-05-08,2025-05-08,"Fallecimiento abuelita, se regaló 2 días y el ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105547,2025-05-11,2025-05-13,3.0,1.0,0.0,full_working_day,2025-05-14,2025-05-16,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,108418,2025-08-18,2025-08-19,2.0,1.0,0.0,full_working_day,2025-08-18,2025-08-19,Compensación de horas por feria informada por ...,...,NaN,NaN,NaN,NaN,4.0,permiso_con_goce,True,per_day,NaN,NaN
230,108485,2025-08-19,2025-08-19,1.0,0.5,0.0,end_working_day,2025-08-19,2025-08-19,,...,NaN,NaN,NaN,NaN,3.0,permiso,False,per_day,NaN,NaN
231,108586,2025-08-12,2025-08-12,1.0,1.0,0.0,full_working_day,2025-08-12,2025-08-12,Permiso por fallecimiento de tío,...,NaN,NaN,NaN,NaN,4.0,permiso_con_goce,True,per_day,NaN,NaN
232,108682,2025-08-21,2025-08-22,2.0,1.0,0.0,full_working_day,2025-08-19,2025-08-20,Permiso por fallecimiento de madre pt1,...,NaN,NaN,NaN,NaN,8.0,permiso_fallecimiento_legal,True,per_day,NaN,NaN


In [ ]:
# === GUARDAR DATOS EXTRAÍDOS EN MYSQL ===
import mysql.connector
import json
from datetime import datetime, date

# Verificar que tenemos datos para guardar
if 'df_combinado' not in globals() or df_combinado.empty:
    print("❌ ERROR: No hay datos para guardar en MySQL")
    print("💡 Ejecuta primero las celdas de extracción de datos")
else:
    print("💾 GUARDANDO DATOS DE BUK EN MYSQL")
    print("=" * 50)
    
    try:
        # 1. CONECTAR A LA BASE DE DATOS
        print("🗄️ Conectando a la base de datos...")
        
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="_Cramer2025_",
            database="rrhh_app",
            charset='utf8mb4',  # Para soportar caracteres especiales
            autocommit=False    # Control manual de commits
        )
        cursor = connection.cursor()
        
        print("✅ Conexión exitosa a la base de datos MySQL")
        print(f"📊 Datos a procesar: {len(df_combinado):,} registros")
        
        # 2. PROCESAR DATOS POR TIPO DE ENDPOINT
        total_insertados = 0
        total_actualizados = 0
        total_errores = 0
        
        # Agrupar datos por tipo de endpoint
        if '_source_endpoint' in df_combinado.columns:
            endpoints_procesados = df_combinado['_source_endpoint'].unique()
            print(f"📋 Endpoints encontrados: {', '.join(endpoints_procesados)}")
            
            for endpoint_type in endpoints_procesados:
                print(f"\n🔄 Procesando: {endpoint_type.upper()}")
                print("-" * 30)
                
                # Filtrar datos del endpoint actual
                df_endpoint = df_combinado[df_combinado['_source_endpoint'] == endpoint_type]
                print(f"📊 Registros a procesar: {len(df_endpoint)}")
                
                insertados_endpoint = 0
                actualizados_endpoint = 0
                errores_endpoint = 0
                
                # ===== PROCESAR INASISTENCIAS =====
                if endpoint_type == 'inasistencias':
                    for index, row in df_endpoint.iterrows():
                        try:
                            # Extraer campos principales
                            absence_id = row.get('id')
                            if pd.isna(absence_id) or absence_id == '':
                                print(f"⚠️ Registro sin ID en fila {index}")
                                errores_endpoint += 1
                                continue
                            
                            # Verificar si ya existe
                            check_sql = "SELECT id FROM inasistencias WHERE absence_id = %s"
                            cursor.execute(check_sql, (absence_id,))
                            exists = cursor.fetchone()
                            
                            # Preparar datos para insertar/actualizar
                            data_values = {
                                'absence_id': absence_id,
                                'employee_id': row.get('employee_id'),
                                'user_first_name': row.get('user_first_name', ''),
                                'user_last_name': row.get('user_last_name', ''),
                                'user_rut': row.get('user_rut', ''),
                                'absence_type_name': row.get('absence_type_name', ''),
                                'start_date': row.get('start_date'),
                                'end_date': row.get('end_date'),
                                'working_days': row.get('working_days'),
                                'application_date': row.get('application_date'),
                                'status': row.get('status', ''),
                                'reason': row.get('reason', ''),
                                'created_at_buk': row.get('created_at'),
                                'updated_at_buk': row.get('updated_at'),
                                'extracted_at': row.get('_fecha_extraccion')
                            }
                            
                            if exists:
                                # ACTUALIZAR registro existente
                                update_sql = """
                                UPDATE inasistencias SET
                                    employee_id = %s,
                                    user_first_name = %s,
                                    user_last_name = %s,
                                    user_rut = %s,
                                    absence_type_name = %s,
                                    start_date = %s,
                                    end_date = %s,
                                    working_days = %s,
                                    application_date = %s,
                                    status = %s,
                                    reason = %s,
                                    created_at_buk = %s,
                                    updated_at_buk = %s,
                                    extracted_at = %s,
                                    updated_at_local = %s
                                WHERE absence_id = %s
                                """
                                
                                update_values = (
                                    data_values['employee_id'],
                                    data_values['user_first_name'],
                                    data_values['user_last_name'], 
                                    data_values['user_rut'],
                                    data_values['absence_type_name'],
                                    data_values['start_date'],
                                    data_values['end_date'],
                                    data_values['working_days'],
                                    data_values['application_date'],
                                    data_values['status'],
                                    data_values['reason'],
                                    data_values['created_at_buk'],
                                    data_values['updated_at_buk'],
                                    data_values['extracted_at'],
                                    datetime.now(),
                                    absence_id
                                )
                                
                                cursor.execute(update_sql, update_values)
                                actualizados_endpoint += 1
                                
                            else:
                                # INSERTAR nuevo registro
                                insert_sql = """
                                INSERT INTO inasistencias (
                                    absence_id, employee_id, user_first_name, user_last_name,
                                    user_rut, absence_type_name, start_date, end_date,
                                    working_days, application_date, status, reason,
                                    created_at_buk, updated_at_buk, extracted_at,
                                    created_at_local, updated_at_local
                                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                                """
                                
                                insert_values = (
                                    data_values['absence_id'],
                                    data_values['employee_id'],
                                    data_values['user_first_name'],
                                    data_values['user_last_name'],
                                    data_values['user_rut'],
                                    data_values['absence_type_name'],
                                    data_values['start_date'],
                                    data_values['end_date'],
                                    data_values['working_days'],
                                    data_values['application_date'],
                                    data_values['status'],
                                    data_values['reason'],
                                    data_values['created_at_buk'],
                                    data_values['updated_at_buk'],
                                    data_values['extracted_at'],
                                    datetime.now(),
                                    datetime.now()
                                )
                                
                                cursor.execute(insert_sql, insert_values)
                                insertados_endpoint += 1
                                
                        except Exception as e:
                            errores_endpoint += 1
                            print(f"❌ Error procesando inasistencia {absence_id}: {e}")
                            continue
                
                # ===== PROCESAR LICENCIAS =====
                elif endpoint_type == 'licencias':
                    for index, row in df_endpoint.iterrows():
                        try:
                            licence_id = row.get('id')
                            if pd.isna(licence_id) or licence_id == '':
                                errores_endpoint += 1
                                continue
                            
                            # Verificar existencia
                            check_sql = "SELECT id FROM licencias WHERE licence_id = %s"
                            cursor.execute(check_sql, (licence_id,))
                            exists = cursor.fetchone()
                            
                            # Preparar datos
                            data_values = {
                                'licence_id': licence_id,
                                'employee_id': row.get('employee_id'),
                                'user_first_name': row.get('user_first_name', ''),
                                'user_last_name': row.get('user_last_name', ''),
                                'licence_type_name': row.get('licence_type_name', ''),
                                'start_date': row.get('start_date'),
                                'end_date': row.get('end_date'),
                                'working_days': row.get('working_days'),
                                'status': row.get('status', ''),
                                'created_at_buk': row.get('created_at'),
                                'updated_at_buk': row.get('updated_at'),
                                'extracted_at': row.get('_fecha_extraccion')
                            }
                            
                            if exists:
                                # Actualizar licencia
                                update_sql = """
                                UPDATE licencias SET
                                    employee_id = %s, user_first_name = %s, user_last_name = %s,
                                    licence_type_name = %s, start_date = %s, end_date = %s,
                                    working_days = %s, status = %s, updated_at_buk = %s,
                                    extracted_at = %s, updated_at_local = %s
                                WHERE licence_id = %s
                                """
                                
                                cursor.execute(update_sql, (
                                    data_values['employee_id'], data_values['user_first_name'],
                                    data_values['user_last_name'], data_values['licence_type_name'],
                                    data_values['start_date'], data_values['end_date'],
                                    data_values['working_days'], data_values['status'],
                                    data_values['updated_at_buk'], data_values['extracted_at'],
                                    datetime.now(), licence_id
                                ))
                                actualizados_endpoint += 1
                            else:
                                # Insertar nueva licencia
                                insert_sql = """
                                INSERT INTO licencias (
                                    licence_id, employee_id, user_first_name, user_last_name,
                                    licence_type_name, start_date, end_date, working_days,
                                    status, created_at_buk, updated_at_buk, extracted_at,
                                    created_at_local, updated_at_local
                                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                                """
                                
                                cursor.execute(insert_sql, (
                                    data_values['licence_id'], data_values['employee_id'],
                                    data_values['user_first_name'], data_values['user_last_name'],
                                    data_values['licence_type_name'], data_values['start_date'],
                                    data_values['end_date'], data_values['working_days'],
                                    data_values['status'], data_values['created_at_buk'],
                                    data_values['updated_at_buk'], data_values['extracted_at'],
                                    datetime.now(), datetime.now()
                                ))
                                insertados_endpoint += 1
                                
                        except Exception as e:
                            errores_endpoint += 1
                            print(f"❌ Error procesando licencia {licence_id}: {e}")
                            continue
                
                # ===== PROCESAR PERMISOS =====
                elif endpoint_type == 'permisos':
                    for index, row in df_endpoint.iterrows():
                        try:
                            permission_id = row.get('id')
                            if pd.isna(permission_id) or permission_id == '':
                                errores_endpoint += 1
                                continue
                            
                            # Verificar existencia
                            check_sql = "SELECT id FROM permisos WHERE permission_id = %s"
                            cursor.execute(check_sql, (permission_id,))
                            exists = cursor.fetchone()
                            
                            # Preparar datos
                            data_values = {
                                'permission_id': permission_id,
                                'employee_id': row.get('employee_id'),
                                'user_first_name': row.get('user_first_name', ''),
                                'user_last_name': row.get('user_last_name', ''),
                                'permission_type_name': row.get('permission_type_name', ''),
                                'start_date': row.get('start_date'),
                                'end_date': row.get('end_date'),
                                'status': row.get('status', ''),
                                'reason': row.get('reason', ''),
                                'created_at_buk': row.get('created_at'),
                                'updated_at_buk': row.get('updated_at'),
                                'extracted_at': row.get('_fecha_extraccion')
                            }
                            
                            if exists:
                                # Actualizar permiso
                                update_sql = """
                                UPDATE permisos SET
                                    employee_id = %s, user_first_name = %s, user_last_name = %s,
                                    permission_type_name = %s, start_date = %s, end_date = %s,
                                    status = %s, reason = %s, updated_at_buk = %s,
                                    extracted_at = %s, updated_at_local = %s
                                WHERE permission_id = %s
                                """
                                
                                cursor.execute(update_sql, (
                                    data_values['employee_id'], data_values['user_first_name'],
                                    data_values['user_last_name'], data_values['permission_type_name'],
                                    data_values['start_date'], data_values['end_date'],
                                    data_values['status'], data_values['reason'],
                                    data_values['updated_at_buk'], data_values['extracted_at'],
                                    datetime.now(), permission_id
                                ))
                                actualizados_endpoint += 1
                            else:
                                # Insertar nuevo permiso
                                insert_sql = """
                                INSERT INTO permisos (
                                    permission_id, employee_id, user_first_name, user_last_name,
                                    permission_type_name, start_date, end_date, status, reason,
                                    created_at_buk, updated_at_buk, extracted_at,
                                    created_at_local, updated_at_local
                                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                                """
                                
                                cursor.execute(insert_sql, (
                                    data_values['permission_id'], data_values['employee_id'],
                                    data_values['user_first_name'], data_values['user_last_name'],
                                    data_values['permission_type_name'], data_values['start_date'],
                                    data_values['end_date'], data_values['status'],
                                    data_values['reason'], data_values['created_at_buk'],
                                    data_values['updated_at_buk'], data_values['extracted_at'],
                                    datetime.now(), datetime.now()
                                ))
                                insertados_endpoint += 1
                                
                        except Exception as e:
                            errores_endpoint += 1
                            print(f"❌ Error procesando permiso {permission_id}: {e}")
                            continue
                
                # Mostrar resultados del endpoint
                print(f"✅ {endpoint_type}: {insertados_endpoint} insertados, {actualizados_endpoint} actualizados, {errores_endpoint} errores")
                
                # Sumar a totales
                total_insertados += insertados_endpoint
                total_actualizados += actualizados_endpoint
                total_errores += errores_endpoint
            
            # 3. CONFIRMAR TRANSACCIÓN
            connection.commit()
            
            print(f"\n" + "=" * 50)
            print(f"✅ PROCESO COMPLETADO EXITOSAMENTE")
            print(f"📊 RESUMEN FINAL:")
            print(f"   • Total insertados: {total_insertados}")
            print(f"   • Total actualizados: {total_actualizados}")
            print(f"   • Total errores: {total_errores}")
            print(f"   • Fecha de procesamiento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"   • Período de datos: {info_extraccion.get('fecha_inicio', 'N/A')} a {info_extraccion.get('fecha_fin', 'N/A')}")
            
        else:
            print("❌ ERROR: Los datos no tienen información de endpoint de origen")
            print("💡 Verifica que la extracción se haya realizado correctamente")
            
    except mysql.connector.Error as e:
        print(f"❌ Error de base de datos MySQL: {e}")
        if 'connection' in locals():
            connection.rollback()
            print("🔄 Transacción revertida")
    
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        print(f"   Tipo: {type(e).__name__}")
        if 'connection' in locals():
            connection.rollback()
            print("🔄 Transacción revertida")
    
    finally:
        # Cerrar conexiones siempre
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals() and connection.is_connected():
            connection.close()
            print("🔌 Conexión a MySQL cerrada")

ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.

In [28]:
# === EXPORTAR df_combinado A EXCEL (SOLUCIONANDO PROBLEMA DE TIMEZONE) ===
import os
from datetime import datetime
import pandas as pd

# Verificar que el DataFrame existe
if 'df_combinado' in globals() and not df_combinado.empty:
    print("📊 EXPORTANDO df_combinado A EXCEL (CORRIGIENDO TIMEZONES)")
    print("=" * 60)
    
    try:
        # Configurar nombre del archivo
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        titulo = "Reporte_API_BUK"
        nombre_archivo = f"{titulo}_{timestamp}.xlsx"
        
        # Crear ruta completa
        ruta_base = r"C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python\Solicitud API\Archivos generados"
        
        # Verificar si la carpeta existe, si no crearla
        if not os.path.exists(ruta_base):
            os.makedirs(ruta_base)
            print(f"📁 Carpeta creada: {ruta_base}")
        
        ruta_completa = os.path.join(ruta_base, nombre_archivo)
        
        print(f"📄 Archivo: {nombre_archivo}")
        print(f"📁 Ubicación: {ruta_base}")
        
        # 🔧 PASO 1: CREAR COPIA DEL DATAFRAME PARA NO AFECTAR EL ORIGINAL
        df_para_exportar = df_combinado.copy()
        
        print(f"\n📋 Información antes de procesar:")
        print(f"   • Filas: {len(df_para_exportar):,}")
        print(f"   • Columnas: {len(df_para_exportar.columns)}")
        
        # 🔧 PASO 2: DETECTAR Y CONVERTIR COLUMNAS CON TIMEZONE
        columnas_fecha_convertidas = 0
        columnas_problematicas = []
        
        print(f"\n🕐 Procesando columnas de fecha/hora...")
        
        for columna in df_para_exportar.columns:
            # Verificar si la columna contiene datos datetime
            if df_para_exportar[columna].dtype == 'datetime64[ns, UTC]' or \
               (hasattr(df_para_exportar[columna].dtype, 'tz') and df_para_exportar[columna].dtype.tz is not None):
                
                print(f"   🔄 Convirtiendo: {columna}")
                try:
                    # Convertir a timezone naive (sin zona horaria)
                    if hasattr(df_para_exportar[columna].dt, 'tz_localize'):
                        # Si tiene timezone, convertir a naive
                        df_para_exportar[columna] = df_para_exportar[columna].dt.tz_localize(None)
                    elif hasattr(df_para_exportar[columna].dt, 'tz_convert'):
                        # Si tiene timezone, convertir a naive
                        df_para_exportar[columna] = df_para_exportar[columna].dt.tz_convert(None)
                    
                    columnas_fecha_convertidas += 1
                    print(f"      ✅ {columna} convertida exitosamente")
                    
                except Exception as e:
                    columnas_problematicas.append(columna)
                    print(f"      ⚠️ Problema con {columna}: {e}")
                    
                    # Intentar conversión alternativa
                    try:
                        df_para_exportar[columna] = pd.to_datetime(df_para_exportar[columna], errors='coerce', utc=False)
                        print(f"      🔄 Conversión alternativa exitosa para {columna}")
                    except:
                        # Como último recurso, convertir a string
                        df_para_exportar[columna] = df_para_exportar[columna].astype(str)
                        print(f"      📝 Convertido a texto: {columna}")
        
        # También revisar columnas object que podrían ser fechas
        for columna in df_para_exportar.select_dtypes(include=['object']).columns:
            # Verificar si parece una fecha
            if any(keyword in columna.lower() for keyword in ['date', 'time', 'created', 'updated']):
                muestra = df_para_exportar[columna].dropna().head(3)
                if len(muestra) > 0 and any('T' in str(val) or '-' in str(val) for val in muestra):
                    try:
                        print(f"   🔍 Verificando columna objeto: {columna}")
                        df_para_exportar[columna] = pd.to_datetime(df_para_exportar[columna], errors='coerce', utc=False)
                        # Si se convirtió exitosamente y tiene timezone, quitarla
                        if hasattr(df_para_exportar[columna].dtype, 'tz') and df_para_exportar[columna].dtype.tz is not None:
                            df_para_exportar[columna] = df_para_exportar[columna].dt.tz_localize(None)
                        columnas_fecha_convertidas += 1
                        print(f"      ✅ {columna} procesada como fecha")
                    except:
                        pass  # Si no se puede convertir, dejarlo como está
        
        print(f"\n📊 Resumen de conversiones:")
        print(f"   • Columnas de fecha convertidas: {columnas_fecha_convertidas}")
        if columnas_problematicas:
            print(f"   • Columnas con problemas: {len(columnas_problematicas)}")
            for col in columnas_problematicas:
                print(f"     - {col}")
        
        # 🔧 PASO 3: EXPORTAR A EXCEL
        print(f"\n💾 Exportando a Excel...")
        df_para_exportar.to_excel(ruta_completa, index=False, engine='openpyxl')
        
        # 🔧 PASO 4: VERIFICAR ÉXITO
        if os.path.exists(ruta_completa):
            tamaño_archivo = os.path.getsize(ruta_completa) / 1024**2  # MB
            print(f"\n✅ ¡EXPORTACIÓN EXITOSA!")
            print(f"   📄 Archivo: {nombre_archivo}")
            print(f"   📊 Registros exportados: {len(df_para_exportar):,}")
            print(f"   📋 Columnas exportadas: {len(df_para_exportar.columns)}")
            print(f"   📊 Tamaño del archivo: {tamaño_archivo:.2f} MB")
            print(f"   📁 Ubicación: {ruta_completa}")
            
            # Mostrar información adicional
            print(f"\n📈 DETALLES DEL ARCHIVO:")
            if '_source_endpoint' in df_para_exportar.columns:
                print(f"   • Distribución por endpoint:")
                conteo = df_para_exportar['_source_endpoint'].value_counts()
                for endpoint, cantidad in conteo.items():
                    print(f"     - {endpoint.capitalize()}: {cantidad:,} registros")
                    
        else:
            print(f"❌ Error: No se pudo crear el archivo")
            
    except PermissionError:
        print(f"❌ ERROR DE PERMISOS:")
        print(f"   • El archivo puede estar abierto en Excel")
        print(f"   • Cierra Excel y prueba nuevamente")
        print(f"   • O cambia la ubicación del archivo")
        
        # Intentar en el escritorio como alternativa
        try:
            print(f"\n🔄 Intentando guardar en el Escritorio...")
            escritorio = os.path.join(os.path.expanduser("~"), "Desktop")
            ruta_escritorio = os.path.join(escritorio, nombre_archivo)
            df_para_exportar.to_excel(ruta_escritorio, index=False, engine='openpyxl')
            print(f"✅ Guardado en el Escritorio: {ruta_escritorio}")
        except Exception as e2:
            print(f"❌ También falló en el Escritorio: {e2}")
        
    except Exception as e:
        print(f"❌ ERROR INESPERADO: {e}")
        print(f"   Tipo: {type(e).__name__}")
        
        # Opción de respaldo: exportar como CSV
        try:
            print(f"\n🔄 Intentando exportar como CSV...")
            nombre_csv = f"{titulo}_{timestamp}.csv"
            ruta_csv = os.path.join(ruta_base, nombre_csv)
            df_para_exportar.to_csv(ruta_csv, index=False, encoding='utf-8-sig')
            print(f"✅ Archivo CSV creado exitosamente: {ruta_csv}")
        except Exception as e3:
            print(f"❌ También falló el CSV: {e3}")

else:
    print("❌ ERROR: df_combinado no existe o está vacío")
    print("💡 Ejecuta primero las celdas de extracción de datos")

📊 EXPORTANDO df_combinado A EXCEL (CORRIGIENDO TIMEZONES)
📄 Archivo: Reporte_API_BUK_20250826_114028.xlsx
📁 Ubicación: C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python\Solicitud API\Archivos generados

📋 Información antes de procesar:
   • Filas: 234
   • Columnas: 37

🕐 Procesando columnas de fecha/hora...
   🔍 Verificando columna objeto: created_at
      ✅ created_at procesada como fecha
   🔍 Verificando columna objeto: updated_at
      ✅ updated_at procesada como fecha

📊 Resumen de conversiones:
   • Columnas de fecha convertidas: 2

💾 Exportando a Excel...

✅ ¡EXPORTACIÓN EXITOSA!
   📄 Archivo: Reporte_API_BUK_20250826_114028.xlsx
   📊 Registros exportados: 234
   📋 Columnas exportadas: 37
   📊 Tamaño del archivo: 0.04 MB
   📁 Ubicación: C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python\Solicitud API\Archivos generados\Reporte_API_BUK_20250826_114028.xlsx

📈 DETALLES DEL ARCHIVO:
   • Distribución por endpoint:
     - Licencias: 162 registros
     - Inas